## Ideal Vacation Spots

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as me
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Retrieve Weather Data From CSV

In [9]:
# reat weather data from created csv file from weather analysis
weather_csv = 'output_data/city_weather_data.csv'
weather_df = me.read_csv(weather_csv)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country
0,Ushuaia,-54.8000,-68.3000,35.26,75,75,14.97,AR
1,Nanortalik,60.1432,-45.2371,42.39,96,100,5.50,GL
2,Vila Velha,-20.3297,-40.2925,78.75,75,75,10.36,BR
3,Tianpeng,30.9866,103.9393,74.14,76,29,4.43,CN
4,Bahir Dar,11.5936,37.3908,62.02,92,100,1.52,ET


### Humidity Heatmap

In [20]:
# configure gmapse with API key
gmaps.configure(api_key=g_key)

# convert humity to floating value and store
# drop any NaN responses if present
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lat and long into variable
locs = weather_df[['Lat', 'Lng']].astype(float)

# create humidity heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, dissipating=False, max_intensity=max(weather_df['Humidity'],
                             point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### DataFrame Creation Based on Ideal Weather Conditions

In [24]:
# narrow dataframe to find ideal weather coniditions
ideal_weather_df = weather_df

# drop any rows that do not contain all three of the following coniditons (ideal weather)
# max temp no more than 80° but no less than 70°
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Max Temp'] <= 80) & (ideal_weather_df['Max Temp'] >= 70)]

# wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed'] < 10)]

# zero cloud cover
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloud Cover'] == 0)]

# drop rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# display dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country
59,Saint Anthony,45.0205,-93.2180,78.12,43,0,8.05,US
97,Cortez,37.3489,-108.5859,79.70,49,0,6.91,US
177,Fukuechō,32.6881,128.8419,77.09,79,0,4.74,JP
234,Guozhen,34.3659,107.3590,78.15,82,0,4.41,CN
319,Winslow,35.0242,-110.6974,78.98,53,0,5.75,US
397,San Ramon,37.7799,-121.9780,76.28,62,0,8.05,US
421,Campos Altos,-19.6961,-46.1714,77.76,29,0,5.77,BR
462,Vila,42.0304,-8.1588,74.70,51,0,8.05,PT
477,Steamboat Springs,40.4850,-106.8317,73.35,51,0,4.61,US


### Hotel Maps

In [30]:
# create hotels dataframe
hotels_df = ideal_weather_df

# main dictionary for parameters of search
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

# iterate rows through each location in dataframe, max 10 locations
for r in 10:
    for index, row in hotel_df.iterrows():
        # get lat and lng from dataframe
        lat = row['Lat']
        lng = row['Lng']
    
        # update location of each iteration
        params['location'] = f'{lat}, {lng}'
    
        # base url for google apis
        base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
        # make API request using call url
        response = requests.get(base_url, params=params).json()
    
        # retrieve results
        results = response['results']
    
        # attempt to get hotel name from index and add to 'Hotel Name' column
        try:
            print(f"The closest hotel to {row['City']} is {results[0]['name']}.")
            hotels_df.loc[index, 'Hotel Name'] = results[0]['name']
            r += 1
    
        # if unable to locate
        except (KeyError, IndexError):
            print(f'Unable to complete record. Checking next city.')
    
        # print separator
        print(f"-" * 35)

The closest hotel to Saint Anthony is Holiday Inn Express Roseville-St. Paul.
-----------------------------------
The closest hotel to Cortez is Sand Canyon National 9 Inn.
-----------------------------------
The closest hotel to Fukuechō is Goto Guesthouse Utojuku & SA Car Rental.
-----------------------------------
The closest hotel to Guozhen is Yucai Hotel.
-----------------------------------
The closest hotel to Winslow is Best Western Plus Winslow Inn.
-----------------------------------
The closest hotel to San Ramon is San Ramon Marriott.
-----------------------------------
The closest hotel to Campos Altos is Hotel São José.
-----------------------------------
The closest hotel to Vila is Hotel Castrum Villae by Walk Hotels.
-----------------------------------
The closest hotel to Steamboat Springs is Steamboat Grand.
-----------------------------------


In [31]:
# show dataframe results 
hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country,Hotel Name
59,Saint Anthony,45.0205,-93.2180,78.12,43,0,8.05,US,Holiday Inn Express Roseville-St. Paul
97,Cortez,37.3489,-108.5859,79.70,49,0,6.91,US,Sand Canyon National 9 Inn
177,Fukuechō,32.6881,128.8419,77.09,79,0,4.74,JP,Goto Guesthouse Utojuku & SA Car Rental
234,Guozhen,34.3659,107.3590,78.15,82,0,4.41,CN,Yucai Hotel
319,Winslow,35.0242,-110.6974,78.98,53,0,5.75,US,Best Western Plus Winslow Inn
397,San Ramon,37.7799,-121.9780,76.28,62,0,8.05,US,San Ramon Marriott
421,Campos Altos,-19.6961,-46.1714,77.76,29,0,5.77,BR,Hotel São José
462,Vila,42.0304,-8.1588,74.70,51,0,8.05,PT,Hotel Castrum Villae by Walk Hotels
477,Steamboat Springs,40.4850,-106.8317,73.35,51,0,4.61,US,Steamboat Grand


In [35]:
# show hotel locations on previously generated heatmap

# template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# store the dataframe rows
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# create hotel symbol layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# add marker layer to heatmap and display
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))